In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import pandas as pd
import urllib.request
import math
import gpytorch
from matplotlib import pyplot as plt
import random
import numpy as np
import urllib.request
import os
from math import floor
from tqdm import tqdm
from blackbox_preaudit import Net

/mnt/infonas/data/tnandy/fa_ant/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
num_device = 0
device = torch.device(f'cuda:{num_device}' if torch.cuda.is_available() else 'cpu')

In [3]:
from vae import VAE

In [4]:
vae1 = VAE(102, 64, 16)
checkpoint = torch.load('checkpoints/vae1.pth')
vae1.load_state_dict(checkpoint['model_state_dict'])
vae1.to(device)
vae1.eval();

vae0 = VAE(102, 64, 16)
checkpoint = torch.load('checkpoints/vae0.pth')
vae0.load_state_dict(checkpoint['model_state_dict'])
vae0.to(device)
vae0.eval();

In [5]:
device

device(type='cuda', index=0)

In [6]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [7]:
data_dim = 102

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train(train_x, train_y):
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(train_x, train_y, likelihood)
    model.to(device)
    likelihood.to(device)
    training_iterations = 60
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    iterator = range(training_iterations)
    model.train()
    likelihood.train()
    for i in range(training_iterations):
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x.to(device))
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y.to(device))
        loss.backward()
        optimizer.step()
    return model, likelihood

In [8]:
def sample_and_discard(database, sample_size):
    samples = database.sample(sample_size)
    database.drop(index=samples.index, inplace=True)
    database.reset_index(drop=True, inplace=True)
    return samples, database

In [9]:
def ignite_pd(dataframe, split=False):
    if split:
        x,y = dataframe.drop(columns=dataframe.columns[-1]).values, dataframe[dataframe.columns[-1]].values
        return torch.Tensor(x.astype(np.float32)).to(device), torch.Tensor(y.astype(np.float32)).to(device)
    else:
        return torch.tensor(dataframe.values.astype(np.float32)).to(device)

In [15]:
blackbox = Net()
blackbox.load_state_dict(torch.load('checkpoints/blackbox_preaudit.pth', map_location=device))
blackbox.to(device)

Net(
  (fc1): Linear(in_features=102, out_features=50, bias=True)
  (act): ReLU()
  (fc2): Linear(in_features=50, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [21]:
df_base = pd.read_csv("../Data/data1.csv")
df_base.drop(['Unnamed: 0'], axis=1, inplace = True)

df_base_pos = df_base[df_base['gender_Female'] > 0]
df_base_neg = df_base[df_base['gender_Female'] <= 0]

df_base_neg = df_base_neg.reset_index(drop=True).copy().drop(columns='income')
df_base_pos = df_base_pos.reset_index(drop=True).copy().drop(columns='income')

sample_pos, df_base_pos = sample_and_discard(df_base_pos, 1000)
pos_queried = ignite_pd(sample_pos)
pos_labels = blackbox(pos_queried)

# sample_neg, df_base_neg = sample_and_discard(df_base_neg, 1000)
# neg_queried = ignite_pd(sample_neg)
# neg_labels = blackbox(neg_queried)

In [17]:
model_neg, likelihood_neg = train(
    torch.tensor(neg_queried),
    torch.tensor(neg_labels).flatten()
)

model_neg.eval()
likelihood_neg.eval()

/tmp/ipykernel_954517/520628438.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(neg_queried),
/tmp/ipykernel_954517/520628438.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(neg_labels).flatten()


GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [18]:
candidates = []
for epoch_outer in tqdm(range(1, 4001)):
    x0_random = torch.normal(0.,1.,size=(1,101), requires_grad=True)
    optimizer0 = torch.optim.AdamW((x0_random,), lr=10)
    best_loss = 10e5
    count = 0
    losses = []
    for epoch in range(1,100):
        optimizer0.zero_grad()
        x0_samples, _, _ = vae1(torch.concatenate([x0_random, torch.Tensor([[0.0]])], axis=1).to(device))
        var0 = likelihood_neg(model_neg(x0_samples.to(device))).variance
        loss = -var0
        loss.backward()
        optimizer0.step()
        if loss < best_loss:
            best_loss = loss
            count = 0
            losses.append(loss)
        else:
            count += 1
        if count  == 5:
            break
    candidates.append(torch.concatenate([x0_random.detach().clone(), torch.Tensor([[0.0]])], axis=1))
    
    if epoch_outer % 500 == 0:
        new_vals = torch.concatenate(candidates)
        new_queries, _, _ = vae1(new_vals.to(device))
        new_labels = blackbox(new_queries)
        neg_queried = torch.concatenate([neg_queried, new_queries])
        neg_labels = torch.concatenate([neg_labels, new_labels])
        model_neg, likelihood_neg = train(
            neg_queried.detach().clone(),
            neg_labels.detach().clone().flatten())
        model_neg.eval()
        likelihood_neg.eval()
        candidates = []

100%|███████████████████████████████████████████████████████████████████████████████| 4000/4000 [34:14<00:00,  1.95it/s]


In [22]:
neg_queried.shape

torch.Size([5000, 102])

In [23]:
#### training the GP ####
model_pos, likelihood_pos = train(
    torch.tensor(pos_queried),
    torch.tensor(pos_labels).flatten())
model_pos.eval()
likelihood_pos.eval()

/tmp/ipykernel_954517/4059754406.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pos_queried),
/tmp/ipykernel_954517/4059754406.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pos_labels).flatten())


GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [24]:
candidates = []
for epoch_outer in tqdm(range(1, 4001)):
    x1_random = torch.normal(0.,1.,size=(1,101), requires_grad=True)
    optimizer1 = torch.optim.AdamW((x1_random,), lr=10)
    best_loss = 10e5
    count = 0
    losses = []
    for epoch in range(1,100):
        optimizer1.zero_grad()
        x1_samples, _, _ = vae1(torch.concatenate([x1_random, torch.Tensor([[1.0]])], axis=1).to(device))
        var1 = likelihood_pos(model_pos(x1_samples.to(device))).variance
        loss = -var1
        loss.backward()
        optimizer1.step()
        if loss < best_loss:
            best_loss = loss
            count = 0
            losses.append(loss)
        else:
            count += 1
        if count  == 5:
            break
    candidates.append(torch.concatenate([x1_random.detach().clone(), torch.Tensor([[1.0]])], axis=1))
    
    if epoch_outer % 500 == 0:
        new_vals = torch.concatenate(candidates)
        new_queries, _, _ = vae1(new_vals.to(device))
        new_labels = blackbox(new_queries)
        pos_queried = torch.concatenate([pos_queried, new_queries])
        pos_labels = torch.concatenate([pos_labels, new_labels])
        model_pos, likelihood_pos = train(
            torch.tensor(pos_queried),
            torch.tensor(pos_labels).flatten())
        model_pos.eval()
        likelihood_pos.eval()
        candidates = []

 12%|█████████▉                                                                      | 499/4000 [04:20<30:21,  1.92it/s]/tmp/ipykernel_954517/712994024.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pos_queried),
/tmp/ipykernel_954517/712994024.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pos_labels).flatten())
100%|███████████████████████████████████████████████████████████████████████████████| 4000/4000 [38:08<00:00,  1.75it/s]


In [27]:
estimated_parity = torch.abs(pos_labels.mean() - neg_labels.mean()).item()

In [31]:
X_pos = ignite_pd(df_base[df_base['gender_Female'] == 1].drop(columns='income'))
X_neg = ignite_pd(df_base[df_base['gender_Female'] == 0].drop(columns='income'))

In [33]:
actual_parity = torch.abs(blackbox(X_pos).mean() - blackbox(X_neg).mean()).item()

In [34]:
print(estimated_parity, actual_parity)

0.0012248754501342773 0.007420867681503296


In [38]:
print("error = ", np.round(np.abs(estimated_parity-actual_parity)*100/actual_parity), "%")

error =  83.0 %
